In [1]:
import gym
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import namedtuple, deque
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64
GAMMA = 0.995
TAU = 1e-3
LR = 5e-4
TOTAL_EPISODES = 2000
MAX_TIME_STEPS = 1000
EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 0.995

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [4]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, seed):
        super().__init__()
        self.seed = torch.manual_seed(seed)
        hidden_layer_1=36
        hidden_layer_2=36
        self.fc1 = nn.Linear(state_size, hidden_layer_1)
        self.fc2 = nn.Linear(hidden_layer_1, hidden_layer_2)
        self.out = nn.Linear(hidden_layer_2, action_size)

    def forward(self, state):
        t = F.relu(self.fc1(state))
        t = F.relu(self.fc2(t))
        actions = self.out(t)
        return actions

In [5]:
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size, seed):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(
            device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(
            device)
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

In [6]:
class Agent():
    def __init__(self, state_size, action_size, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, seed)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)
        
        self.t_step += 1
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

    def select_action(self, state, eps=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        future = self.qnetwork_target.forward(next_states).detach().max(dim=1)[0].unsqueeze(1)
        target = rewards + gamma * future * (1 - dones)
        predict = self.qnetwork_local.forward(states).gather(1, actions)
        loss = F.mse_loss(predict, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Target network update
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)

    def soft_update(self, local_model, target_model, tau):
        """
        θ_target = τ*θ_local + (1 - τ)*θ_target
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [7]:
def plot(rewards_per_episode):
    plt.figure(figsize=(7,4))
    plt.title('Reward')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.plot(rewards_per_episode)
    plt.plot(moving_average(rewards_per_episode))
    plt.pause(0.001)
    print("Episode\t", len(rewards_per_episode))
    print("Current reward\t", rewards_per_episode[-1])
        
def moving_average(values, period=50):
    values = torch.tensor(values, dtype=torch.float)
    if len(values) >= period:
        moving_avg = values.unfold(dimension=0, size=period, step=1).mean(dim=1).flatten(start_dim=0)
        moving_avg = torch.cat((torch.zeros(period-1), moving_avg))
        return moving_avg.numpy()
    else:
        moving_avg = torch.zeros(len(values))
        return moving_avg.numpy()

In [8]:
env = gym.make('LunarLander-v2')
env.seed(0)
agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.n, seed=0)

total_episodes = TOTAL_EPISODES
max_time_steps = MAX_TIME_STEPS
eps_start = EPS_START
eps_end = EPS_END
eps_decay = EPS_DECAY

rewards_per_episode = []

eps = eps_start
for episode in range(1, total_episodes + 1):
    state = env.reset()
    total_reward = 0
    for timestep in range(max_time_steps):
        env.render('human')
        action = agent.select_action(state, eps)
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        if done:
            break
    eps = max(eps_end, eps * eps_decay)

    rewards_per_episode.append(total_reward)
    plot(rewards_per_episode)
    print("Steps\t", timestep)
    print("Epsilon\t", eps)
    display.clear_output(wait=True)
    if total_reward >= 200:
        date_time = datetime.now().strftime("%y/%m/%d_%H:%M:%S")
        FILE = "model_" + date_time + ".pth"
        torch.save(agent.qnetwork_local.state_dict(), FILE)
        break

KeyboardInterrupt: 